<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Notebook_Clustering_KMeans_PySpark_Con_Conclusiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering con K-Means en PySpark

Este notebook demuestra cómo aplicar el algoritmo K-Means utilizando PySpark MLlib con datos sintéticos. Incluye generación de datos, entrenamiento del modelo, evaluación y visualización.

In [ ]:
# 1. Inicialización de PySpark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KMeansExample") \
    .getOrCreate()

In [ ]:
# 2. Generar datos sintéticos (usando sklearn)
from sklearn.datasets import make_blobs
import pandas as pd

features, _ = make_blobs(n_samples=300, centers=3, cluster_std=1.0, random_state=42)
df_pd = pd.DataFrame(features, columns=["x", "y"])
df_pd.head()

In [ ]:
# 3. Convertir a Spark DataFrame
df_spark = spark.createDataFrame(df_pd)
df_spark.show(5)

In [ ]:
# 4. VectorAssembler para transformar columnas en vector de características
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=["x", "y"], outputCol="features")
df_features = vec_assembler.transform(df_spark)
df_features.select("features").show(5, truncate=False)

In [ ]:
# 5. Aplicar KMeans
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=3, seed=1, featuresCol="features")
model = kmeans.fit(df_features)
predictions = model.transform(df_features)
predictions.select("x", "y", "prediction").show(10)

In [ ]:
# 6. Evaluar WCSS (Within Set Sum of Squared Errors)
wcss = model.summary.trainingCost
print(f"WCSS (Suma de errores al cuadrado): {wcss:.2f}")

In [ ]:
# 7. Visualización de los clusters
import matplotlib.pyplot as plt

pdf = predictions.select("x", "y", "prediction").toPandas()
plt.figure(figsize=(8, 6))
for cluster in sorted(pdf['prediction'].unique()):
    subset = pdf[pdf['prediction'] == cluster]
    plt.scatter(subset['x'], subset['y'], label=f"Cluster {cluster}")
plt.title("Clustering con K-Means en PySpark")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

## 📌 Conclusiones

- El algoritmo K-Means puede implementarse de forma eficiente con PySpark, aprovechando su arquitectura distribuida.
- La visualización mostró que los puntos se agrupan correctamente en tres clusters, lo que valida la precisión del modelo en este conjunto de datos sintético.
- La métrica WCSS permite evaluar la compacidad de los clusters y se puede usar para comparar configuraciones con diferentes valores de k.
- Este ejercicio ilustra claramente la utilidad de Spark MLlib para tareas de clustering en escenarios reales de gran escala, donde la eficiencia del procesamiento es clave.
- La integración de PySpark con herramientas de visualización permite enriquecer el análisis y mejorar la comprensión de los resultados por parte de los analistas de datos.